In [2]:

import numpy as np
import matplotlib.pyplot as plt
import sys
import math
from math import log, ceil
import dimod
from dwave.cloud import Client
import dwave.inspector
from dwave.samplers import SimulatedAnnealingSampler
from dwave.system.samplers import DWaveSampler 
from dwave.system.composites import EmbeddingComposite 
from dwave.system import LeapHybridCQMSampler
from dimod import ConstrainedQuadraticModel
import statistics
from functools import reduce
import os
import time
from datetime import datetime

In [3]:
# Início da execução
start_time = time.time()

# Funcao para criar o CQM (Constraint Quadratic Model) para o problema das N-Rainhas
def n_queens_cqm(N):
    # Cria o objeto CQM
    cqm = ConstrainedQuadraticModel()

    # Variaveis binarias x[i, j] para cada linha i e coluna j (1 se uma rainha for colocada, 0 caso contrario)
    # Isso cria N*N variaveis binarias. Cada variavel x[i, j] eh 1 se uma rainha for colocada na linha i e coluna j, e 0 caso contrario.
    x = {}
    for i in range(N):
        for j in range(N):
            # Criando uma variavel binaria para cada posicao no tabuleiro de xadrez
            x[(i, j)] = dimod.Binary(f'x_{i}_{j}')  # f'x_{i}_{j}' eh o rotulo da variavel

    # **Restricao 1: Uma rainha por linha**
    # Para cada linha i, impomos a restricao de que deve haver exatamente uma rainha naquela linha.
    for i in range(N):
        # Somando sobre todas as colunas na linha i, exigimos que a soma de x[i, j] para todos os j seja exatamente 1
        # Isso garante que exatamente uma rainha seja colocada em cada linha.
        cqm.add_constraint(sum(x[(i, j)] for j in range(N)) == 1, label=f"row_{i}")\

    # **Restricao 2: Uma rainha por coluna**
    # Da mesma forma, para cada coluna j, impomos a restricao de que deve haver exatamente uma rainha naquela coluna.
    for j in range(N):
        # Somando sobre todas as linhas na coluna j, exigimos que a soma de x[i, j] para todos os i seja exatamente 1
        # Isso garante que exatamente uma rainha seja colocada em cada coluna.
        cqm.add_constraint(sum(x[(i, j)] for i in range(N)) == 1, label=f"column_{j}")

    # **Restricao 3: Nenhuma rainha na mesma diagonal** (diagonais positivas e negativas)
    # Agora adicionamos restricoes para impedir que duas rainhas estejam na mesma diagonal.
    
    # Para as restricoes das diagonais, verificamos ambas as direcoes: 
    # - Diagonais que inclinam para cima (\ direcao), onde (i - j) eh constante.
    # - Diagonais que inclinam para baixo (/ direcao), onde (i + j) eh constante.
    
    for i in range(N):
        for j in range(N):
            # Verificar conflitos na direcao \ (inclinacao negativa).
            for k in range(i + 1, N):
                # Verificar a celula (i, j) com as celulas na mesma diagonal.
                # Diagonal 1 (\ direcao): Se duas rainhas compartilharem o mesmo (i - j), elas estao na mesma diagonal.
                # Precisamos garantir que apenas uma rainha seja colocada nessas diagonais.
                
                # Garantir que nao haja duas rainhas compartilhando a mesma diagonal na direcao \.
                if j + (k - i) < N:
                    # Adiciona uma restricao para garantir que no maximo uma rainha seja colocada na diagonal \.
                    # Isso eh feito somando as variaveis naquela diagonal e garantindo que a soma seja no maximo 1.
                    cqm.add_constraint(x[(i, j)] + x[(k, j + (k - i))] <= 1, label=f"diag1_{i}_{j}_{k}")
                
                # Verificar a direcao / (inclinacao positiva).
                # Diagonal 2 (/ direcao): Se duas rainhas compartilharem o mesmo (i + j), elas estao na mesma diagonal.
                if j - (k - i) >= 0:
                    # Adiciona uma restricao para garantir que no maximo uma rainha seja colocada na diagonal /.
                    cqm.add_constraint(x[(i, j)] + x[(k, j - (k - i))] <= 1, label=f"diag2_{i}_{j}_{k}")
                    
    # Retorna o modelo CQM criado
    return cqm

In [4]:
# Parâmetros
N = 8
time_limit_set = 5
plot_images = True

# Criação e resolução do modelo
cqm = n_queens_cqm(N)
sampler = LeapHybridCQMSampler()
sampleset = sampler.sample_cqm(cqm, time_limit=time_limit_set, label=f'{N}-Queens Problem')

In [5]:
# Filtra soluções viáveis
feasible_sampleset = sampleset.filter(lambda row: row.is_feasible)

# Coletar soluções únicas
unique_solutions = set()
unique_positions_list = []

if len(feasible_sampleset):
    print(f"Número total de soluções viáveis encontradas: {len(feasible_sampleset)}\n")

    for sample in feasible_sampleset.samples():
        # sample já é um dicionário com as variáveis
        positions = tuple(sorted(
            (int(var.split('_')[1]), int(var.split('_')[2]))
            for var, val in sample.items() if val == 1
        ))

        if positions not in unique_solutions:
            unique_solutions.add(positions)
            unique_positions_list.append(positions)

    print(f"Número de soluções viáveis únicas: {len(unique_positions_list)}\n")

    # Agora imprimimos somente as únicas
    for idx, positions in enumerate(unique_positions_list, 1):
        print(f"Solução única #{idx}:")
        print("Posições das rainhas (linha, coluna):")
        for pos in positions:
            print(pos)

        print("\nTabuleiro:")
        for i in range(N):
            row = ['Q' if (i, j) in positions else '.' for j in range(N)]
            print(' '.join(row))
        print("\n" + "-" * 40 + "\n")

else:
    print("Nenhuma solução viável encontrada.")


Número total de soluções viáveis encontradas: 55

Número de soluções viáveis únicas: 24

Solução única #1:
Posições das rainhas (linha, coluna):
(0, 3)
(1, 7)
(2, 4)
(3, 2)
(4, 0)
(5, 6)
(6, 1)
(7, 5)

Tabuleiro:
. . . Q . . . .
. . . . . . . Q
. . . . Q . . .
. . Q . . . . .
Q . . . . . . .
. . . . . . Q .
. Q . . . . . .
. . . . . Q . .

----------------------------------------

Solução única #2:
Posições das rainhas (linha, coluna):
(0, 2)
(1, 4)
(2, 7)
(3, 3)
(4, 0)
(5, 6)
(6, 1)
(7, 5)

Tabuleiro:
. . Q . . . . .
. . . . Q . . .
. . . . . . . Q
. . . Q . . . .
Q . . . . . . .
. . . . . . Q .
. Q . . . . . .
. . . . . Q . .

----------------------------------------

Solução única #3:
Posições das rainhas (linha, coluna):
(0, 4)
(1, 0)
(2, 7)
(3, 3)
(4, 1)
(5, 6)
(6, 2)
(7, 5)

Tabuleiro:
. . . . Q . . .
Q . . . . . . .
. . . . . . . Q
. . . Q . . . .
. Q . . . . . .
. . . . . . Q .
. . Q . . . . .
. . . . . Q . .

----------------------------------------

Solução única #4:
Posições

In [6]:
# Metadados
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d_%Hh%M")
folder_name = f"run_{N}queens_{timestamp}"
os.makedirs(folder_name, exist_ok=True)

# Arquivos de saída
file_all = os.path.join(folder_name, "solucoes_viaveis.txt")
file_unique = os.path.join(folder_name, "solucoes_unicas.txt")
file_heatmap = os.path.join(folder_name, "heatmap_ocupacao.png")

# Inicializações
unique_solutions = set()
unique_positions_list = []
heatmap = np.zeros((N, N))

# Escrita dos arquivos
with open(file_all, "w", encoding="utf-8") as fall, open(file_unique, "w", encoding="utf-8") as funique:
    fall.write(f"Parâmetros da execução:\n")
    fall.write(f"N = {N}\n")
    fall.write(f"Tempo limite do solver = {time_limit_set} s\n")
    fall.write(f"Data/Hora = {timestamp}\n")
    fall.write(f"Solver usado = {sampler.solver.name}\n\n")

    if len(feasible_sampleset):
        fall.write(f"Número total de soluções viáveis encontradas: {len(feasible_sampleset)}\n\n")

        for count, sample in enumerate(feasible_sampleset.data(), 1):
            sample_dict = sample.sample
            energy = sample.energy

            positions = tuple(sorted(
                (int(var.split('_')[1]), int(var.split('_')[2]))
                for var, val in sample_dict.items() if val == 1
            ))

            # Escreve todas as soluções
            fall.write(f"Solução #{count}:\n")
            fall.write(f"Energy: {energy}\n")
            fall.write("Posições das rainhas:\n")
            for pos in positions:
                fall.write(f"{pos}\n")
            fall.write("Tabuleiro:\n")
            for i in range(N):
                row = ['Q' if (i, j) in positions else '.' for j in range(N)]
                fall.write(' '.join(row) + '\n')
            fall.write("-" * 40 + "\n\n")

            # Armazena únicas
            if positions not in unique_solutions:
                unique_solutions.add(positions)
                unique_positions_list.append((positions, energy))

                # Atualiza heatmap
                for i, j in positions:
                    heatmap[i, j] += 1        

        funique.write(f"Número de soluções viáveis únicas: {len(unique_positions_list)}\n\n")
        for idx, (positions, energy) in enumerate(unique_positions_list, 1):
            funique.write(f"Solução única #{idx}:\n")
            funique.write(f"Energy: {energy}\n")
            funique.write("Posições das rainhas:\n")
            for pos in positions:
                funique.write(f"{pos}\n")
            funique.write("Tabuleiro:\n")
            for i in range(N):
                row = ['Q' if (i, j) in positions else '.' for j in range(N)]
                funique.write(' '.join(row) + '\n')
            funique.write("-" * 40 + "\n\n")

            # Geração de imagem da solução
# Geração de imagem da solução
            if plot_images:
                fig, ax = plt.subplots(figsize=(6, 6))
                ax.set_xlim(-0.5, N - 0.5)
                ax.set_ylim(-0.5, N - 0.5)
                ax.axis("off")
            
                # Fundo xadrez
                for i in range(N):
                    for j in range(N):
                        color = '#f0d9b5' if (i + j) % 2 == 0 else '#b58863'
                        rect = plt.Rectangle((j - 0.5, N - 1 - i - 0.5), 1, 1, facecolor=color)
                        ax.add_patch(rect)
            
                # Posiciona as rainhas
                for i, j in positions:
                    ax.text(j, N - 1 - i, '♛', ha='center', va='center', fontsize=30, color='black')
            
                # Adiciona nomes das colunas (A, B, C, ...)
                col_labels = [chr(ord('A') + j) for j in range(N)]
                for j, label in enumerate(col_labels):
                    ax.text(j, -1, label, ha='center', va='center', fontsize=12)
            
                # Adiciona nomes das linhas (1, 2, ..., N)
                for i in range(N):
                    ax.text(-1, i, str(N - i), ha='center', va='center', fontsize=12)
            
                ax.set_title(f"Solução {idx}", fontsize=14, pad=10)
                plt.tight_layout()
                plt.savefig(os.path.join(folder_name, f"tabuleiro_{idx}.png"))
                plt.close()


# Geração do heatmap
if plot_images and np.sum(heatmap) > 0:
    plt.figure(figsize=(6, 5))
    plt.imshow(heatmap, cmap='hot', interpolation='nearest')
    plt.colorbar(label='Frequência de ocupação')
    plt.title(f"Heatmap de Ocupação - {N} Rainhas")
    plt.xlabel("Colunas")
    plt.ylabel("Linhas")
    plt.xticks(np.arange(N))
    plt.yticks(np.arange(N))
    plt.savefig(file_heatmap)
    plt.close()

# Tempo total
end_time = time.time()
duration = end_time - start_time

# Adiciona tempo de execução ao final dos arquivos
with open(file_all, "a", encoding="utf-8") as fall:
    fall.write(f"Tempo total de execução: {duration:.2f} segundos\n")
with open(file_unique, "a", encoding="utf-8") as funique:
    funique.write(f"Tempo total de execução: {duration:.2f} segundos\n")